## Identifying Ascending Triangles in SP500 Index Data

### Introduction:
Welcome to the project on "Identifying Ascending Triangles in SP500 Index Data". In this project, we will be working with a dataset containing more than 12,000 bars or candles of the SP500 index, with basic information such as the TimeStamp, Open, High, Low, and Close, for each hour, for 2 years from January 1, 2021, to December 31, 2022.

The aim of this project is to develop a model that can detect Ascending Triangles in the SP500 index data. An Ascending Triangle is a bullish chart pattern that forms when there is a series of higher lows that form a trendline, and a horizontal resistance level is tested multiple times but not broken. When the price breaks above the resistance level, it is often followed by a significant price increase.

We will be using Python and various libraries such as Pandas, Plotly, and NumPy to preprocess the data and build the Ascending Triangle detection model. We will then visualize the identified patterns using candlestick charts and annotations.

By the end of this project, we will have a better understanding of Ascending Triangles and how to detect them in financial data using Python.

#### Step 1: Importing Required Libraries

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Step 2: Load and display the dataset

In [3]:

# Here i'm using Excel data becuase CSV data have some issues when i'm import the data.
# So that i converted data into Excel.


df = pd.read_excel('DataHistoSP.xlsx', index_col='Time')

In [4]:
# This function displays the first 5 rows of the DataFrame 'df'. 

df.head()

,Open,High,Low,Close
Time,,,,
2021.01.02 23:00,3733.00,3733.00,3733.00,3733.00
2021.01.03 19:00,3733.00,3733.00,3733.00,3733.00
2021.01.03 22:00,3733.00,3734.00,3733.00,3733.00
2021.01.03 23:00,3733.00,3739.75,3726.75,3733.25
2021.01.04 00:00,3733.25,3733.50,3722.50,3724.75


#### Step 3: Checking the dataset having missing values

In [5]:


missing_values = df.isnull().sum()
print(missing_values)

Open     0
High     0
Low      0
Close    0
dtype: int64


#### Step 4: Drop weekends

In [6]:
# (Saturday's and Sunday's) beacause the market is off on these days

df = df[pd.DatetimeIndex(df.index).dayofweek < 5]

#### Step 5: Calculate the Volume of the Candles

This function calculates the volume of a single candle based on the High, Low, and Close prices. It first calculates the volume-weighted average price (VWAP) of the candle, then multiplies it by the range of prices (High minus Low) to get the candle volume. This function assumes that the input data contains columns named 'High', 'Low', and 'Close'.

Volume = (High - Low) * ((High + Low + Close) / 3)


In [8]:
def calculate_candle_volume(row):
    vwap = (row['High'] + row['Low'] + row['Close']) / 3
    volume = (row['High'] - row['Low']) * vwap
    return volume

In [9]:
# Apply the calculate_candle_volume() function to each row of the DataFrame and store the result in a new column
df['Volume'] = df.apply(calculate_candle_volume, axis=1)

In [22]:
df.head()

,Open,High,Low,Close,Volume
Time,,,,,
2021.01.04 00:00,3733.25,3733.50,3722.50,3724.75,40996.083333
2021.01.04 01:00,3724.75,3733.75,3724.25,3731.75,35434.208333
2021.01.04 02:00,3732.00,3735.50,3730.50,3731.25,18662.083333
2021.01.04 03:00,3731.25,3735.00,3730.75,3733.75,15865.958333
2021.01.04 04:00,3733.75,3735.25,3731.00,3732.50,15864.895833


#### Step 3: Plot the dataset in window by window in Candlestick

This code creates a candlestick chart using Plotly to visualize the price data in windows of 40 candles at a time. The `df` DataFrame should contain columns named 'Open', 'High', 'Low', and 'Close' with the corresponding price data, as well as a DatetimeIndex to represent the timestamps of each candle. 
The for loop iterates through the DataFrame in increments of 40 candles, creating a `window` DataFrame for each iteration. The `go.Candlestick` function is used to create a candlestick trace for each window, and these traces are added to the `fig` figure. 
The resulting chart shows the price data in a sliding window view, with each window offset by 40 candles from the previous one. The chart also includes a volume bar chart for each window to show the trading volume of each candle. The chart title is set to 'Dataset in Window by Window', and the x-axis range slider is disabled to prevent accidental zooming.


In [11]:

fig = go.Figure()

for i in range(0, len(df), 40):
    window = df.iloc[i:i+40]
    fig.add_trace(go.Candlestick(x=window.index,
                                  open=window['Open'],
                                  high=window['High'],
                                  low=window['Low'],
                                  close=window['Close'],
                                  name='Window ' + str(i)))


fig.update_layout(title='Dataset in Window by Window',
                  xaxis_rangeslider_visible=False)
fig.show()

#### Step 4: Detect Ascending Triangle Pattern from the data

This code defines a function to detect an ascending triangle pattern from the above data. The function takes a pandas DataFrame as input and checks if the DataFrame contains an ascending triangle pattern by identifying a series of higher lows that form an ascending trendline, and then checking if this trendline intersects with a horizontal resistance line.
If an ascending triangle pattern is detected, the function plots the candlestick chart with the identified pattern and returns the DataFrame with an additional column that indicates whether the pattern was detected or not. If no ascending triangle pattern is detected, the function returns the original DataFrame with the additional column indicating no pattern detected.


In [16]:


def detect_ascending_triangle(df):
    # Initialize counter variable
    count = 0
    
    # Check if there are any NaN or infinite values in the data
    if not np.all(np.isfinite(df.values)):
        return df.assign(ascending_triangle=False)
    
    # Check if the pattern has a series of higher lows that form an ascending trendline
    lows = df['Low']
    trendline = lows == lows.sort_values().values
    
    # Check if the trendline has at least 3 points
    if trendline.sum() < 3:
        return df.assign(ascending_triangle=False)
    
    # Calculate the trendline by fitting a linear regression model
    x = range(len(df))
    y = df['Low']
    slope, intercept = np.polyfit(x, y, 1)
    trendline = slope * x + intercept
    
    # Calculate the horizontal line by finding the maximum high value in the DataFrame
    resistance = max(df['High'])
    
    # Detect the ascending triangle by checking if the trendline intersects with the resistance line
    if trendline[-1] >= resistance:
        # Increment counter and print message
        count += 1
        print(f"Ascending triangle detected (#{count})")
        
        # Create a subplot grid with one row and one column
        fig = make_subplots(rows=1, cols=1)
        
        # Add the candlestick trace to the grid
        fig.add_candlestick(x=df.index,
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            row=1,
                            col=1)
        
        # Add the trendline, horizontal line, and annotations as additional traces to the figure
        fig.add_trace(go.Scatter(x=df.index, y=trendline, mode='lines', line=dict(color='green')), row=1, col=1)
        fig.add_trace(go.Scatter(x=df.index, y=[resistance]*len(df), mode='lines', line=dict(color='red')), row=1, col=1)
        
        top_x = [df.index[0], df.index[-1]]
        top_y = [resistance, resistance]
        bottom_x = [df.index[0], df.index[-1]]
        bottom_y = [df['Low'][0], df['Low'][-1]]
        fig.add_trace(go.Scatter(x=top_x, y=top_y, mode='markers', marker=dict(symbol='triangle-up', color='blue')), row=1, col=1)
        fig.add_trace(go.Scatter(x=bottom_x, y=bottom_y, mode='markers', marker=dict(symbol='triangle-down', color='blue')), row=1, col=1)
        
        fig.update_layout(title='Ascending Triangle Detected')
        fig.show()
        
        return df.assign(ascending_triangle=True)
    
    else:
        return df.assign(ascending_triangle=False)

#### Step 5: Detection and Visualization of Ascending Triangles from Above Dataset

This code is used to detect and visualize ascending triangles from Above Dataset. The code uses the function `detect_ascending_triangle` to detect ascending triangles in windows of 40 data points. The function returns a Boolean column indicating if an ascending triangle was detected in the window. The code then iterates through the windows of data, and if an ascending triangle is detected, it increments a counter and adds the ascending triangle to a `go.Figure` object using `go.Candlestick`. Finally, the code updates the layout of the plot to display the total number of ascending triangles detected and shows the plot using `fig.show()`.

In [18]:


# Initialize variables
count = 0
fig = go.Figure()

# Iterate through the windows of data
for i in range(0, len(df), 40):
    # Get the window of data and detect the ascending triangle
    window = df.iloc[i:i+40]
    window = detect_ascending_triangle(window)
    
    # If an ascending triangle is detected, increment the counter and add it to the plot
    if window['ascending_triangle'].iloc[-1]:
        count += 1
        fig.add_trace(go.Candlestick(x=window.index,
                                      open=window['Open'],
                                      high=window['High'],
                                      low=window['Low'],
                                      close=window['Close'],
                                      name=f"Ascending Triangle #{count}"))

# Update the layout of the plot
fig.update_layout(title=f"Total number of ascending triangles detected: {count}")
fig.show()


Ascending triangle detected (#1)


Ascending triangle detected (#1)


Ascending triangle detected (#1)


Ascending triangle detected (#1)


Ascending triangle detected (#1)


Ascending triangle detected (#1)


Ascending triangle detected (#1)


Ascending triangle detected (#1)


#### Conclusion: 
Based on the analysis of the SP500 index from January 1, 2021 to December 31, 2022, we have developed a small pattern identification model that detects Ascending Triangles. The model was able to identify a total of 8 number of ascending triangles in the dataset. This information can be useful for traders and investors who are looking for potential bullish signals in the market. Further research and analysis can be conducted to improve the accuracy and effectiveness of the model.